# Welcome to the Kotlin Notebook!

## Basics

Let's start with something simple: execute a cell with a simple arithmetics and immediately see the result.

In [ ]:
2 + 2

Execution results could be saved in variables and reused

In [ ]:
val result = 3 * 14

In [ ]:
result

In [ ]:
(Out[3] as Int) / 2

Variables defined in the notebook can lose their nullability if they are not actually nulls

In [ ]:
val a1: Int = 1
val a2: Int? = 2
val a3: Int? = null

In [ ]:
a1 + a2 // OK, a2 was converted to Int

In [ ]:
a1 + a3 // compile-time error

## Rich outputs

Outputs might be not only plain text. They could also be images and HTML. HTML can contain CSS and JavaScript.

In [ ]:
HTML("""
    <p>Counter: <span id="ctr">0</span> <button onclick="inc()">Increment</button></p>
    <script>
      function inc() {
        let counter = document.getElementById("ctr")
        counter.innerHTML = parseInt(counter.innerHTML) + 1;
      }
    </script>
""")

**NB!** If your outputs contain JS, notebook should be marked as trusted.

![trusted-notebook](screenshots/screenshot3.png)

Images could be loaded by link. In this case, it won't show if the link breaks or if you lose Internet connection

In [ ]:
%use lib-ext(0.11.0-398)

In [ ]:
Image("https://kotlinlang.org/docs/images/kotlin-logo.png", embed = false).withWidth(300)

You can also embed images. In this case they will stay in the notebook forever

In [ ]:
val kotlinMascot = Image("https://blog.jetbrains.com/wp-content/uploads/2023/04/DSGN-16174-Blog-post-banner-and-promo-materials-for-post-about-Kotlin-mascot_3.png", embed = true).withWidth(400)
kotlinMascot

The cell can also have several outputs, to achieve it use `DISPLAY()` function

In [ ]:
DISPLAY(HTML("<h2>Kodee is back!</h2>"))
DISPLAY(kotlinMascot)

With Kotlin Notebook, you can also render LaTeX formulae

In [ ]:
LATEX("c^2 = a^2 + b^2 - 2 a b \\cos\\alpha")

You can also output BufferedImage's. They are embedded into notebook

In [ ]:
import java.awt.Color
import java.awt.image.BufferedImage

val width = 300
val height = width

val image = BufferedImage(width, height, BufferedImage.TYPE_INT_ARGB)

val graphics = image.createGraphics()
graphics.background = Color.BLACK
graphics.clearRect(0, 0, width, height)
graphics.setRenderingHint(
    java.awt.RenderingHints.KEY_ANTIALIASING,
    java.awt.RenderingHints.VALUE_ANTIALIAS_ON
)
graphics.color = Color.WHITE
graphics.fillRect(width / 10, height * 8 / 10, width * 10 / 20, height / 10)
graphics.dispose()

image

Generally, you can display any output using `mimeResult` function.
We're using [Jupyter approach](https://docs.jupyter.org/en/latest/reference/mimetype.html) for outputs.

In [ ]:
mimeResult(
    MimeTypes.PLAIN_TEXT to "JetBrains logo",
    MimeTypes.HTML to "<b>JetBrains</b> logo"
)

## Using libraries and dependencies

You can always turn on source and binary dependencies of a current project.
To do it, use the corresponding button in the toolbar.

In [ ]:
import java.io.File
import javax.imageio.ImageIO

fun showScreenshot(id: Any) {
    DISPLAY(ImageIO.read(File("screenshots/screenshot$id.png")))
}

In [ ]:
showScreenshot(1)

It is also possible to set these options for the newly created notebooks

In [ ]:
showScreenshot(2)

Of course, you can depend on various JVM libraries even if you don't have a project in the current scope.
The simpliest option we offer is to use predefined library descriptors, you can find which are available using `:help` command or [here](https://github.com/Kotlin/kotlin-jupyter#list-of-supported-libraries).

In [ ]:
:help

Let's try `kotlinx.serialization` library

In [ ]:
%use serialization

It allows us to serialize and deserialize classes.

In [ ]:
import kotlinx.serialization.Serializable

@Serializable
class User(val firstName: String, val lastName: String)

In [ ]:
val bob = User("Alex", "Green")

In [ ]:
Json { prettyPrint = true }.encodeToString(bob)

It is possible to specify descriptors' and underlying libraries' versions, write and contribute your own descriptors and much more.
You can read about it [here](https://github.com/Kotlin/kotlin-jupyter/blob/master/docs/libraries.md)

Also, you can add dependencies for any Maven libraries you want

In [ ]:
USE {
    repositories {
        // Any additional repositories. Maven central is already included
        // maven("<url>")
    }
    dependencies {
        // Here we add kandy plotting library
        implementation("org.jetbrains.kotlinx:kandy-lets-plot:0.4.3")
    }

    // Sometimes library integration are loaded transitively and you don't want them to do it.
    discardIntegrationTypeNameIf {
        it.startsWith("org.jetbrains.kotlinx.dataframe.")
    }
}

## Renderers and other integration features

Let's try to conduct an experiment: we'll throw 50 dices and count the sum of points on them.
Then, we'll repeat this experiment some reasonable number of times and plot the distribution using kandy library we've just loaded.

In [ ]:
import kotlin.random.Random

fun diceNTimesSum(n: Int): Int {
    return (1..n).sumOf { Random.nextInt(1, 7) }
}

val experimentData = (1..100000).map { diceNTimesSum(50) }.groupBy { it }.mapValues { it.value.size }.entries.sortedBy { it.key }
val experimentX = experimentData.map { it.key }
val experimentY = experimentData.map { it.value }

val gaussPlot = plot {
    bars {
        x(experimentX)
        y(experimentY)
    }
}
gaussPlot

In [ ]:
gaussPlot::class

As you can see kandy's `Plot` object was rendered to some image. That's because `kandy` library defines a **renderer** for this type of objects. We also can define renderers ourselves.

In [ ]:
%use dataframe

In [ ]:
val bob = User("Bob", "Brown")
bob

`User` isn't a data class, that's why it was rendered this way.

In [ ]:
USE {
    // Match is based on runtime type here, beware of type erasure
    render<User> { listOf(it).toDataFrame() }
}

In [ ]:
bob

We can also use full syntax for defining renderers. It's verbose but let you do many things

In [ ]:
class User2(val name: String)

In [ ]:
USE {
    addRenderer(object : RendererHandler {
        override fun replaceVariables(mapping: Map<String, String>): RendererHandler {
            return this
        }

        override val execution: ResultHandlerExecution
            get() = ResultHandlerExecution { host, res ->
                FieldValue("Value of ${res.name} is a user with name ${(res.value as User2).name}", null)
            }

        override fun accepts(value: Any?): Boolean {
            return value != null && value::class == User2::class
        }
    })
}

In [ ]:
User2("Felix")

What do the libraries bring except for dependencies and renderers? First of all, default imports.
These imports are implicitly added to all subsequent cells.

In [ ]:
fun loadFileFromGitHub(repositoryUrl: String, filePath: String): String {
    val rawUrl = "$repositoryUrl/raw/master/$filePath"
    val url = URL(rawUrl)
    val connection = url.openConnection()
    connection.setRequestProperty("Accept", "application/vnd.github.v3.raw")

    val inputStream = connection.getInputStream()
    val content = inputStream.bufferedReader().use { it.readText() }

    return content
}

fun loadDescriptor(name: String) {
    val text = loadFileFromGitHub("https://github.com/Kotlin/kotlin-jupyter-libraries", "$name.json")
    DISPLAY(MIME(
        "text/markdown" to "```json\n$text```"
    ))
}

Notice `imports` section in the following descriptor:

In [ ]:
loadDescriptor("kaliningraph")

You might have noticed that almost every descriptor in `Kotlin/kotlin-jupyter-libraries` contains link and description.
They are used to build `:help` command output and [our README](https://github.com/Kotlin/kotlin-jupyter#list-of-supported-libraries)

## Creating an integration

So, we learned the concept of the integration. It is a wrapper on top of the usual Kotlin library that simplifies and elevates the experience of using this library in the notebook.
Comprehensive guide to writing library integrations can be found [here](). To inspire you, there is an example of the `dataframe` integration below.

In [ ]:
%use dataframe

In [ ]:
val df = DataFrame.read("https://raw.githubusercontent.com/mwaskom/seaborn-data/master/iris.csv")

Let's try to investigate the data that we have just read.

In [ ]:
df.petal_length

In [ ]:
df.filter { petal_width >= 1.5 && petal_length < 4.5 }

There is also a special variable in the integration to manage display options. For example, we can limit number of displayed rows.

In [ ]:
dataFrameConfig.display.rowsLimit = 5

In [ ]:
df

Let's now see what's happening there under the hood. First, we enable debugging logging that will print the code that is executed under the hood.

In [ ]:
%trackExecution

Then, we define some other dataframe.

In [ ]:
val df1 = dataFrameOf("A", "B", "C")(1, "Str1", null, 2, "Str2", 3)

As you can see, a marker interface `_DataFrameType1` is created and property accessors are generated for it. Let's check what's the type of `df1` now:

In [ ]:
::df1

So, `df1` now is not simply a `DataFrame<*>`, it's `DataFrame<_DataFrameType1>`. That's exactly what allows us to statically resolve defined property accessors on it.

In [ ]:
%trackExecution off

API that DataFrame uses to achieve all of these is open, and you can try it yourself! The code of the integration is available [here](https://github.com/Kotlin/dataframe/blob/master/core/src/main/kotlin/org/jetbrains/kotlinx/dataframe/jupyter/Integration.kt)

## Inter-cell API

To use Kotlin Notebook API you don't necessarily need to create a separate file with integration. As it was shown above, you can place in `USE { }` call the same integration that you would
place in the standalone library or JSON descriptor. However, notebook offers API to get some information about the current notebook session and to set it up.

The main entry point is `notebook`: it allows you to investigate what cells were already executed, what libraries were loaded, what renderers and variable processors are loaded and gives the ability to unload them.
Read the documentation of the `Notebook` interface for reliable and actual information about this API

In [ ]:
notebook.cellsList.take(5).map { "> " + it.code }.joinToString("\n")

In [ ]:
notebook.kernelVersion

In [ ]:
notebook.cellVariables

Also, a couple of options is available with `SessionOptions` object.

In [ ]:
SessionOptions.resolveSources // could be switched off to speed up dependencies loading process

In [ ]:
SessionOptions.resolveMpp // could be switched on to resolve MPP libraries such as kotlinx-serialization from "universal" Maven coordinates

## Sharing the notebooks

Essentially, Kotlin notebooks are Jupyter notebooks. It means they could be opened, edited and run with any Jupyter client such as [Jupyter Notebook](https://jupyter.org/),
[Jupyter Lab](https://jupyterlab.readthedocs.io/en/latest/) and [Datalore](https://www.jetbrains.com/datalore/). [We plan](https://youtrack.jetbrains.com/issue/KTNB-291/Datalore-Share-action) to integrate with Datalore better in the future.
The only limitation is that project dependencies will be not included into the notebook in other clients. However, you can build the logic based on Jupyter client type:

In [ ]:
if (notebook.jupyterClientType != JupyterClientType.KOTLIN_NOTEBOOK) {
    // load substitutive dependencies
}

Notebooks could be also loaded and viewed on GitHub (including gists). Limitation there is that JS isn't executed in the outputs.
To overcome it in `kandy`, we [add extra SVG output that is hidden from JS](https://github.com/Kotlin/kandy/blob/main/kandy-lets-plot/src/main/kotlin/org/jetbrains/kotlinx/kandy/letsplot/util/rendering.kt#L46).

Kotlin Notebooks could be also converted to some other format using [nbconvert tool](https://nbconvert.readthedocs.io/en/latest/).

## What's next?

Kotlin Notebook project is experimental and actively developed. Use the latest version of plugin to be in sync.
We eagerly need your stories and feedback: share them in the dedicated [#notebooks](https://kotlinlang.slack.com/archives/C05333T208Y) Slack channel or in our [issue tracker](https://youtrack.jetbrains.com/issues/KTNB).

See you in the next cell!